In [1]:
#так называемый плохой способ проверки модели
from sklearn.datasets import load_iris 
iris = load_iris()
X = iris.data
y = iris.target

In [38]:
#классификатор k-средних 
#в данном случае гиперпараметр отвечает за соответсттвие близжайших неизвестных меток
#если при k=1 точность 100% то при k=100 только 66%
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [39]:
model.fit(X, y)
y_model = model.predict(X)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

In [42]:
#хороший способ проверки модели - ОТЛОЖЕННЫЕ ДАННЫЕ
from sklearn.model_selection import train_test_split
#разделяем данные по 50% в каждом наборе
X1, X2, y1, y2 = train_test_split(X, y, random_state =0, test_size=0.5)
#обучение модели на одном из наборов
model.fit(X1, y1)
#оцениаем работу на другом
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)
#при измеенении гиперпараметра n_neighors -> 1000+ будет сильно снижаться точность(логично)
#КОНЕЦ ПРИМЕРА МОДЕЛИ С "СОСЕДЯМИ"

0.9066666666666666

In [46]:
#мето перекрестной проверки для модели
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

(0.96, 0.9066666666666666)

In [101]:
#пятиблочная перекрестная проверка
from sklearn.model_selection import LeaveOneOut

cross_val_score(model, X, y, cv = 5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [105]:
#проверка по отдельным объектам (leave-one-out cross-validation)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [106]:
scores.mean()

0.96

In [104]:
#КОНЕЦ ПРИМЕРА МОДЕЛИ С СОСЕДНИМИ АПРОКСИМИЗАЦИЯМИ